In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

import warnings
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import xgboost as xgb


import eli5
from eli5.sklearn import PermutationImportance
from eli5.permutation_importance import get_score_importances

/Users/morphine/.pyenv/versions/anaconda3-5.3.1/envs/py38/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/morphine/.pyenv/versions/anaconda3-5.3.1/envs/py38/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
import pandas as pd # 데이터 분석
import numpy as np # 행렬 연산, version: 1.6.1

import os
import sys

import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgbm
import sklearn
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import TruncatedSVD, PCA, FastICA, FactorAnalysis, KernelPCA, DictionaryLearning
from sklearn.decomposition import IncrementalPCA, LatentDirichletAllocation,MiniBatchSparsePCA, SparsePCA

import itertools

In [4]:
pd.set_option('max_columns', 100)
pd.set_option('display.precision', 4)

rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
sns.set(font_scale=2.5)

warnings.filterwarnings('ignore')

# 데이터 로드

In [5]:
data_dir = Path('../input/dankook')
sub_dir = Path('../output/')

train_file = data_dir / 'train.csv'
test_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'
sub_file = sub_dir / 'submission.csv'

target_column = 'class'
SEED = 2020

In [6]:
df_train = pd.read_csv(train_file, index_col=0)
df_test = pd.read_csv(test_file, index_col=0)

# 이상치 제거

In [7]:
# test의 MinMax 범위 넘는 행은 train에서 제거
train_shape = df_train.shape[0]

for col in df_train.columns[:18]:
    df_train = df_train.loc[np.logical_and(df_train[col]>=df_test[col].min(),
                            df_train[col]<=df_test[col].max())]

print('제거된 행 개수 :', train_shape - df_train.shape[0])

제거된 행 개수 : 77


In [8]:
df_train.describe()

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
count,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,3.1992e+05,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000
mean,19.8525,18.4318,17.7352,17.3037,17.0603,6.0481e-02,19.6298,18.2591,17.6158,17.2151,16.9944,6.3334,6.1396,1.1758,1.1765,1.1751,1.1754,1.1761,1.1165
std,1.9392,1.6598,1.4586,1.3141,1.3310,2.9110e-01,1.9114,1.6594,1.4672,1.3250,1.3232,8.8817,8.5522,0.1163,0.1181,0.1147,0.1155,0.1171,0.9234
min,2.2651,-12.4441,7.7314,7.7115,-9.2548,-2.5915e+01,-30.6337,-18.6560,-8.7560,-3.6973,0.2159,1.0000,1.0000,1.0001,1.0001,1.0002,1.0002,1.0002,0.0000
25%,18.7244,17.4750,16.8773,16.5240,16.2890,3.5724e-05,18.5639,17.3486,16.7874,16.4532,16.2343,1.0000,1.0000,1.0883,1.0885,1.0878,1.0881,1.0883,0.0000
50%,19.4195,18.1405,17.5259,17.1498,16.9177,4.7153e-02,19.2647,18.0225,17.4434,17.0874,16.8694,2.0000,2.0000,1.1794,1.1792,1.1794,1.1794,1.1793,1.0000
75%,20.4320,19.0728,18.4279,18.0074,17.7288,9.4606e-02,20.1976,18.8883,18.2908,17.9070,17.6555,5.0000,5.0000,1.2275,1.2260,1.2292,1.2286,1.2268,2.0000
max,49.1436,46.3383,45.1299,32.8634,52.6127,4.4620e+01,30.7779,30.6132,31.2940,30.5509,28.5710,44.0000,42.0000,2.0491,2.0786,2.0205,2.0347,2.0637,2.0000


In [9]:
df_test.describe()

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z
count,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,8.0000e+04,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000
mean,19.8598,18.4371,17.7397,17.3065,17.1490,6.0083e-02,19.6350,18.2638,17.6193,17.2178,16.8702,6.3499,6.1578,1.1760,1.1767,1.1753,1.1756,1.1763
std,1.9543,1.7030,1.4786,1.3207,24.6431,3.4684e-01,1.9210,1.6657,1.4760,1.3324,35.4367,8.8728,8.5509,0.1164,0.1182,0.1148,0.1156,0.1173
min,-0.3385,-51.1753,-5.4387,5.3267,-39.5272,-3.0149e+01,-30.6337,-18.6560,-8.7560,-3.6973,-9999.0000,1.0000,1.0000,1.0001,1.0001,1.0002,1.0002,1.0001
25%,18.7273,17.4780,16.8820,16.5255,16.2882,3.3645e-05,18.5709,17.3522,16.7906,16.4562,16.2310,1.0000,1.0000,1.0881,1.0883,1.0877,1.0879,1.0882
50%,19.4261,18.1451,17.5299,17.1560,16.9212,4.7115e-02,19.2674,18.0247,17.4487,17.0912,16.8733,2.0000,2.0000,1.1794,1.1793,1.1794,1.1794,1.1793
75%,20.4344,19.0793,18.4345,18.0080,17.7333,9.4769e-02,20.1999,18.8948,18.2952,17.9109,17.6578,5.0000,5.0000,1.2278,1.2262,1.2294,1.2289,1.2269
max,56.8471,94.3591,46.6913,33.0259,6976.3922,4.6390e+01,30.8899,30.9529,31.6536,30.9478,28.6441,44.0000,42.0000,2.0502,2.0797,2.0216,2.0358,2.0648


# 변수 생성 

In [10]:
for i in ['u','g','r','i','z']:
    df_train[f'ugriz_{i}'] = df_train[i]
    df_test[f'ugriz_{i}'] = df_test[i]
df_train.drop(columns=['u','g','r','i','z'], axis=1, inplace=True)
df_test.drop(columns=['u','g','r','i','z'], axis=1, inplace=True)

In [11]:
ugriz_col = [c for c in df_train.columns if c.find('ugriz') != -1]
dered_col = [c for c in df_train.columns if c.find('dered') != -1]
airmass_col = [c for c in df_train.columns if c.find('airmass') != -1]

In [12]:
# zip 함수를 이용하여 각 Row별, Magnitude별, max, min, max-min, std, sum을 구한다.
for prefix , g in zip(['ugriz','dered','airmass'], [ugriz_col, dered_col, airmass_col]):
    df_train[f'{prefix}_max'] = df_train[g].max(axis=1)
    df_test[f'{prefix}_max'] = df_test[g].max(axis=1)
    
    df_train[f'{prefix}_min'] = df_train[g].min(axis=1)
    df_test[f'{prefix}_min'] = df_test[g].min(axis=1)
    
    df_train[f'{prefix}_diff'] = df_train[f'{prefix}_max'] - df_train[f'{prefix}_min']
    df_test[f'{prefix}_diff'] = df_test[f'{prefix}_max'] - df_test[f'{prefix}_min']
    
    df_train[f'{prefix}_std'] = df_train[g].std(axis=1)
    df_test[f'{prefix}_std'] = df_test[g].std(axis=1)
    
    df_train[f'{prefix}_sum'] = df_train[g].sum(axis=1)
    df_test[f'{prefix}_sum'] = df_test[g].sum(axis=1)

In [13]:
# diff feature 추가 예: z - i
# itertools combinations을 활용하여 전체 magnitude에서 diff를 구함
# 총 105가지 조합이 나옴. 여기서 안 좋은 것은 permutation importance를 활용하여 제거할 예정
diff_feature = []
for c1, c2 in itertools.combinations(ugriz_col[::-1]+dered_col[::-1]
                                     +airmass_col[::-1],2):
    new_c = f'{c1}_{c2}_diff'
    df_train[new_c] = df_train[c1]-df_train[c2]
    df_test[new_c] = df_test[c1]-df_test[c2]
    diff_feature.append(new_c)
    

In [14]:
# 각 maginitude 별 max-max, min-min, sum-sum 을 구함

for c in itertools.combinations(['ugriz','dered','airmass'],2):
    df_train[f'{c[0]}_{c[1]}_max_diff'] = df_train[f'{c[0]}_max'] - df_train[f'{c[1]}_max']
    df_test[f'{c[0]}_{c[1]}_max_diff'] = df_test[f'{c[0]}_max'] - df_test[f'{c[1]}_max']
    
    df_train[f'{c[0]}_{c[1]}_min_diff'] = df_train[f'{c[0]}_min'] - df_train[f'{c[1]}_min']
    df_test[f'{c[0]}_{c[1]}_min_diff'] = df_test[f'{c[0]}_min'] - df_test[f'{c[1]}_min']
    
    df_train[f'{c[0]}_{c[1]}_sum_diff'] = df_train[f'{c[0]}_sum'] - df_train[f'{c[1]}_sum']
    df_test[f'{c[0]}_{c[1]}_sum_diff'] = df_test[f'{c[0]}_sum'] - df_test[f'{c[1]}_sum']

In [15]:
# 아래와 같은 공식들을 구현함
# http://classic.sdss.org/dr4/algorithms/sdssUBVRITransform.html 

def make_2flux_feature(train, test, c1, c2, func, mag_list=None):
    
    for c in mag_list:
        x=train[f'{c}_{c1}'].values
        y=train[f'{c}_{c2}'].values
        train[f'{c}_{func.__name__}'] = func(x,y)
        
        x=test[f'{c}_{c1}'].values
        y=test[f'{c}_{c2}'].values
        test[f'{c}_{func.__name__}'] = func(x,y)
        
def UB_jester(x1,x2):
    return 0.75*(x1-x2)-0.81

make_2flux_feature(df_train,df_test, 'u','g',UB_jester,['ugriz'])

def BV_jester(x1,x2):
    return 0.62*(x1-x2)+0.15

make_2flux_feature(df_train,df_test, 'g','r',BV_jester,['ugriz'])

def VR_jester(x1,x2):
    return 0.38*(x1-x2)+0.27

make_2flux_feature(df_train, df_test, 'r','i',VR_jester,['ugriz'])

def RcIc_jester(x1,x2):
    return 0.72*(x1-x2)+0.27

make_2flux_feature(df_train, df_test, 'r','i', RcIc_jester, ['ugriz'])

def B_jester(x1,x2):
    return x2+0.17*(x1-x2)+0.11

make_2flux_feature(df_train,df_test, 'u','g', B_jester, ['ugriz'])

def V_jester(x1,x2):
    return x1-0.52*(x1-x2)-0.03

make_2flux_feature(df_train,df_test, 'g','r',V_jester, ['ugriz'])

In [16]:
# https://www.sdss.org/dr16/algorithms/segue_target_selection/#Legacy

# 아래 나열된 모든 ugriz 등급은 달리 명시되지 않는 한 dereddened PSF mag입니다.
# dered_X 사용하면 될듯, 일단은 잘 몰라서 ugriz_X 버전과 dered_X 버전을 비교해서 사용하면 될듯.

# ugriz_X 버전

all_data = pd.concat([df_train,df_test], axis=0)
all_data.fillna(-1, inplace=True)

all_data['ugriz_icolor'] = all_data['ugriz_u']*(-0.436) + all_data['ugriz_g']*(1.129) + all_data['ugriz_r']*(-0.119) + all_data['ugriz_i']*(-0.574) + 0.1984

all_data['ugriz_ucolor'] = all_data['ugriz_u']*(-0.249) + all_data['ugriz_g']*(0.794) + all_data['ugriz_r']*(-0.555) + 0.234

all_data['ugriz_p1'] = (all_data['ugriz_u']-all_data['ugriz_g'])*(0.91) + (all_data['ugriz_g']-all_data['ugriz_r'])*(0.415)- 1.280

all_data['ugriz_r_std_div'] = all_data['ugriz_r']/all_data['ugriz_r'].std()

In [17]:
# https://www.sdss.org/dr16/algorithms/legacy_target_selection/
# 멸망과 관련 있는 거라고 함.
# dered_X 가 어떤 곳에서는 멸망과 관련 있는 데이터였음.
# ugriz_X를 붉어짐에 대한 걸로 사용하다면, 여기서는 dered_X를 멸망과 관련 있는 것과 사용하면 될듯

all_data['dered_orthogonal'] = (all_data['dered_r']-all_data['dered_i'])-(all_data['dered_g']-all_data['dered_r'])/4-0.18

all_data['dered_parallel'] = 0.7*(all_data['dered_g']-all_data['dered_r']) + 1.2*((all_data['dered_r']-all_data['dered_i'])-0.18)

In [18]:
# https://www.sdss.org/dr12/algorithms/magnitudes/
# 문서에는 psfMag에 대한 데이터인데, 그것을 나는 ugriz로 사용하고 있으니까 urgiz로 사용

color_list = ['u', 'g', 'r', 'i', 'z']
b_list = [1.4*10e-10, 0.9*10e-10, 1.2*10e-10, 1.8*10e-10, 7.4*10e-10]
f0_list = [24.63, 25.11, 24.80, 24.36, 22.83]
for c, b, f0 in zip(color_list, b_list, f0_list):
    all_data[f'ugriz_{c}_asinh'] = -2.5*np.log(10)*(np.arcsinh((all_data[f'ugriz_{c}']/f0)/(2*b))+np.log(b))

# feature 선택

In [19]:
selected_columns = ['redshift', 'dered_g', 'dered_i', 'dered_z', 'nObserve', 'airmass_u', 'airmass_g', 'airmass_r', 'airmass_i', 'ugriz_r', 'ugriz_i', 'ugriz_z', 'ugriz_max', 'ugriz_diff', 'ugriz_std', 'ugriz_sum', 'dered_max', 'dered_diff', 'dered_sum', 'airmass_max', 'airmass_min', 'airmass_diff', 'airmass_std', 'airmass_sum', 'ugriz_z_ugriz_i_diff', 'ugriz_z_ugriz_r_diff', 'ugriz_z_ugriz_u_diff', 'ugriz_z_dered_z_diff', 'ugriz_z_dered_i_diff', 'ugriz_z_dered_r_diff', 'ugriz_z_dered_g_diff', 'ugriz_z_dered_u_diff', 'ugriz_z_airmass_z_diff', 'ugriz_z_airmass_i_diff', 'ugriz_z_airmass_r_diff', 'ugriz_z_airmass_u_diff', 'ugriz_i_ugriz_u_diff', 'ugriz_i_dered_z_diff', 'ugriz_i_dered_i_diff', 'ugriz_i_dered_g_diff', 'ugriz_i_dered_u_diff', 'ugriz_i_airmass_z_diff', 'ugriz_i_airmass_i_diff', 'ugriz_i_airmass_u_diff', 'ugriz_r_ugriz_g_diff', 'ugriz_r_ugriz_u_diff', 'ugriz_r_dered_i_diff', 'ugriz_r_dered_g_diff', 'ugriz_r_dered_u_diff', 'ugriz_r_airmass_i_diff', 'ugriz_g_ugriz_u_diff', 'ugriz_g_dered_i_diff', 'ugriz_g_dered_r_diff', 'ugriz_g_dered_g_diff', 'ugriz_g_dered_u_diff', 'ugriz_g_airmass_i_diff', 'ugriz_g_airmass_r_diff', 'ugriz_u_dered_z_diff', 'ugriz_u_dered_i_diff', 'ugriz_u_dered_r_diff', 'ugriz_u_dered_g_diff', 'ugriz_u_dered_u_diff', 'ugriz_u_airmass_i_diff', 'ugriz_u_airmass_r_diff', 'ugriz_u_airmass_g_diff', 'ugriz_u_airmass_u_diff', 'dered_z_dered_i_diff', 'dered_z_dered_r_diff', 'dered_z_dered_g_diff', 'dered_z_dered_u_diff', 'dered_z_airmass_i_diff', 'dered_z_airmass_r_diff', 'dered_z_airmass_g_diff', 'dered_i_dered_r_diff', 'dered_i_dered_g_diff', 'dered_i_dered_u_diff', 'dered_i_airmass_z_diff', 'dered_i_airmass_g_diff', 'dered_i_airmass_u_diff', 'dered_r_dered_g_diff', 'dered_r_dered_u_diff', 'dered_r_airmass_i_diff', 'dered_r_airmass_u_diff', 'dered_g_dered_u_diff', 'dered_g_airmass_z_diff', 'dered_g_airmass_r_diff', 'dered_g_airmass_g_diff', 'dered_g_airmass_u_diff', 'dered_u_airmass_z_diff', 'dered_u_airmass_i_diff', 'dered_u_airmass_g_diff', 'airmass_z_airmass_i_diff', 'airmass_i_airmass_r_diff', 'airmass_i_airmass_g_diff', 'airmass_i_airmass_u_diff', 'airmass_r_airmass_g_diff', 'airmass_r_airmass_u_diff', 'airmass_g_airmass_u_diff', 'ugriz_dered_max_diff', 'ugriz_dered_min_diff', 'ugriz_dered_sum_diff', 'ugriz_airmass_max_diff', 'ugriz_airmass_sum_diff', 'dered_airmass_max_diff', 'ugriz_B_jester', 'ugriz_V_jester', 'ugriz_icolor', 'ugriz_ucolor', 'ugriz_p1', 'dered_orthogonal','class']

In [20]:
all_data = all_data[selected_columns].copy()

In [21]:
all_data.shape

(399923, 111)

In [22]:
all_data.describe()

,redshift,dered_g,dered_i,dered_z,nObserve,airmass_u,airmass_g,airmass_r,airmass_i,ugriz_r,ugriz_i,ugriz_z,ugriz_max,ugriz_diff,ugriz_std,ugriz_sum,dered_max,dered_diff,dered_sum,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,ugriz_z_ugriz_i_diff,ugriz_z_ugriz_r_diff,ugriz_z_ugriz_u_diff,ugriz_z_dered_z_diff,ugriz_z_dered_i_diff,ugriz_z_dered_r_diff,ugriz_z_dered_g_diff,ugriz_z_dered_u_diff,ugriz_z_airmass_z_diff,ugriz_z_airmass_i_diff,ugriz_z_airmass_r_diff,ugriz_z_airmass_u_diff,ugriz_i_ugriz_u_diff,ugriz_i_dered_z_diff,ugriz_i_dered_i_diff,ugriz_i_dered_g_diff,ugriz_i_dered_u_diff,ugriz_i_airmass_z_diff,ugriz_i_airmass_i_diff,ugriz_i_airmass_u_diff,ugriz_r_ugriz_g_diff,ugriz_r_ugriz_u_diff,ugriz_r_dered_i_diff,ugriz_r_dered_g_diff,ugriz_r_dered_u_diff,ugriz_r_airmass_i_diff,...,ugriz_u_dered_u_diff,ugriz_u_airmass_i_diff,ugriz_u_airmass_r_diff,ugriz_u_airmass_g_diff,ugriz_u_airmass_u_diff,dered_z_dered_i_diff,dered_z_dered_r_diff,dered_z_dered_g_diff,dered_z_dered_u_diff,dered_z_airmass_i_diff,dered_z_airmass_r_diff,dered_z_airmass_g_diff,dered_i_dered_r_diff,dered_i_dered_g_diff,dered_i_dered_u_diff,dered_i_airmass_z_diff,dered_i_airmass_g_diff,dered_i_airmass_u_diff,dered_r_dered_g_diff,dered_r_dered_u_diff,dered_r_airmass_i_diff,dered_r_airmass_u_diff,dered_g_dered_u_diff,dered_g_airmass_z_diff,dered_g_airmass_r_diff,dered_g_airmass_g_diff,dered_g_airmass_u_diff,dered_u_airmass_z_diff,dered_u_airmass_i_diff,dered_u_airmass_g_diff,airmass_z_airmass_i_diff,airmass_i_airmass_r_diff,airmass_i_airmass_g_diff,airmass_i_airmass_u_diff,airmass_r_airmass_g_diff,airmass_r_airmass_u_diff,airmass_g_airmass_u_diff,ugriz_dered_max_diff,ugriz_dered_min_diff,ugriz_dered_sum_diff,ugriz_airmass_max_diff,ugriz_airmass_sum_diff,dered_airmass_max_diff,ugriz_B_jester,ugriz_V_jester,ugriz_icolor,ugriz_ucolor,ugriz_p1,dered_orthogonal,class
count,3.9992e+05,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,3.9992e+05,3.9992e+05,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,...,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000
mean,6.0401e-02,18.2600,17.2156,16.9696,6.3367,1.1758,1.1766,1.1752,1.1755,17.7361,17.3043,17.0781,19.9045,2.8902,1.1722,90.4052,19.6663,2.7500,89.6925,1.1792,1.1725,6.7180e-03,2.6558e-03,5.8792,-0.2262,-0.6580,-2.7759,0.1085,-0.1375,-0.5384,-1.1820,-2.5528,15.9019,15.9026,15.9029,15.9022,-2.5497,0.3347,0.0887,-0.9558,-2.3265,16.1281,16.1288,16.1285,-0.6968,-2.1178,0.5205,-0.5239,-1.8947,16.5606,...,0.2231,18.6785,18.6788,18.6774,18.6781,-0.2460,-0.6469,-1.2905,-2.6612,15.7941,15.7944,15.7930,-0.4009,-1.0444,-2.4152,16.0394,16.0390,16.0398,-0.6436,-2.0143,16.4410,16.4407,-1.3708,17.0839,17.0849,17.0835,17.0842,18.4546,18.4553,18.4542,0.0007,0.0003,-0.0011,-0.0003,-0.0014,-0.0006,0.0008,0.2382,0.0980,0.7127,18.7253,84.5260,18.4871,18.7844,18.0405,0.3095,0.0825,0.3023,0.0600,0.6931
std,3.0307e-01,1.6606,1.3264,15.8934,8.8799,0.1163,0.1181,0.1147,0.1155,1.4626,1.3155,11.0859,11.1756,11.1193,4.9642,13.0856,1.9262,1

# 데이터셋 분리

In [23]:
# train set
X = all_data.loc[all_data['class'] != -1 , :]
X.drop(columns='class',inplace=True,axis=1)
y = all_data.loc[all_data['class'] != -1, 'class']
y.astype(int)

# test set
test = all_data.loc[all_data['class'] == -1, :]
test.drop(columns='class', inplace=True,axis=1)

# train set split
SEED = 2020
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state = SEED)

In [24]:
df_train.shape, df_test.shape, X.shape, y.shape, test.shape

((319923, 154), (80000, 153), (319923, 110), (319923,), (80000, 110))

In [25]:
X.describe()

,redshift,dered_g,dered_i,dered_z,nObserve,airmass_u,airmass_g,airmass_r,airmass_i,ugriz_r,ugriz_i,ugriz_z,ugriz_max,ugriz_diff,ugriz_std,ugriz_sum,dered_max,dered_diff,dered_sum,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,ugriz_z_ugriz_i_diff,ugriz_z_ugriz_r_diff,ugriz_z_ugriz_u_diff,ugriz_z_dered_z_diff,ugriz_z_dered_i_diff,ugriz_z_dered_r_diff,ugriz_z_dered_g_diff,ugriz_z_dered_u_diff,ugriz_z_airmass_z_diff,ugriz_z_airmass_i_diff,ugriz_z_airmass_r_diff,ugriz_z_airmass_u_diff,ugriz_i_ugriz_u_diff,ugriz_i_dered_z_diff,ugriz_i_dered_i_diff,ugriz_i_dered_g_diff,ugriz_i_dered_u_diff,ugriz_i_airmass_z_diff,ugriz_i_airmass_i_diff,ugriz_i_airmass_u_diff,ugriz_r_ugriz_g_diff,ugriz_r_ugriz_u_diff,ugriz_r_dered_i_diff,ugriz_r_dered_g_diff,ugriz_r_dered_u_diff,ugriz_r_airmass_i_diff,...,ugriz_u_dered_g_diff,ugriz_u_dered_u_diff,ugriz_u_airmass_i_diff,ugriz_u_airmass_r_diff,ugriz_u_airmass_g_diff,ugriz_u_airmass_u_diff,dered_z_dered_i_diff,dered_z_dered_r_diff,dered_z_dered_g_diff,dered_z_dered_u_diff,dered_z_airmass_i_diff,dered_z_airmass_r_diff,dered_z_airmass_g_diff,dered_i_dered_r_diff,dered_i_dered_g_diff,dered_i_dered_u_diff,dered_i_airmass_z_diff,dered_i_airmass_g_diff,dered_i_airmass_u_diff,dered_r_dered_g_diff,dered_r_dered_u_diff,dered_r_airmass_i_diff,dered_r_airmass_u_diff,dered_g_dered_u_diff,dered_g_airmass_z_diff,dered_g_airmass_r_diff,dered_g_airmass_g_diff,dered_g_airmass_u_diff,dered_u_airmass_z_diff,dered_u_airmass_i_diff,dered_u_airmass_g_diff,airmass_z_airmass_i_diff,airmass_i_airmass_r_diff,airmass_i_airmass_g_diff,airmass_i_airmass_u_diff,airmass_r_airmass_g_diff,airmass_r_airmass_u_diff,airmass_g_airmass_u_diff,ugriz_dered_max_diff,ugriz_dered_min_diff,ugriz_dered_sum_diff,ugriz_airmass_max_diff,ugriz_airmass_sum_diff,dered_airmass_max_diff,ugriz_B_jester,ugriz_V_jester,ugriz_icolor,ugriz_ucolor,ugriz_p1,dered_orthogonal
count,3.1992e+05,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,3.1992e+05,3.1992e+05,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,...,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000
mean,6.0481e-02,18.2591,17.2151,16.9944,6.3334,1.1758,1.1765,1.1751,1.1754,17.7352,17.3037,17.0603,19.8847,2.8701,1.1633,90.3835,19.6647,2.7232,89.7141,1.1792,1.1725,6.7146e-03,2.6544e-03,5.8791,-0.2434,-0.6749,-2.7921,0.0659,-0.1547,-0.5554,-1.1988,-2.5694,15.8842,15.8849,15.8852,15.8846,-2.5487,0.3093,0.0887,-0.9554,-2.3261,16.1276,16.1283,16.1279,-0.6966,-2.1173,0.5201,-0.5239,-1.8946,16.5597,...,1.5933,0.2227,18.6770,18.6773,18.6759,18.6767,-0.2206,-0.6213,-1.2647,-2.6353,15.8190,15.8193,15.8179,-0.4007,-1.0441,-2.4147,16.0389,16.0385,16.0393,-0.6434,-2.0140,16.4403,16.4400,-1.3707,17.0830,17.0840,17.0826,17.0833,18.4536,18.4543,18.4532,0.0007,0.0003,-0.0011,-0.0003,-0.0014,-0.0006,0.0008,0.2200,0.0731,0.6694,18.7055,84.5044,18.4855,18.7833,18.0396,0.3094,0.0826,0.3019,0.0599
std,2.9110e-01,1.6594,1.3250,1.3232,8.8817,0.1163,0.1181,0.1147,0.1155,1.4586,1.3141,1.3310,1.9642,1.5576,0.6366,7.118

In [26]:
test.describe()

,redshift,dered_g,dered_i,dered_z,nObserve,airmass_u,airmass_g,airmass_r,airmass_i,ugriz_r,ugriz_i,ugriz_z,ugriz_max,ugriz_diff,ugriz_std,ugriz_sum,dered_max,dered_diff,dered_sum,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,ugriz_z_ugriz_i_diff,ugriz_z_ugriz_r_diff,ugriz_z_ugriz_u_diff,ugriz_z_dered_z_diff,ugriz_z_dered_i_diff,ugriz_z_dered_r_diff,ugriz_z_dered_g_diff,ugriz_z_dered_u_diff,ugriz_z_airmass_z_diff,ugriz_z_airmass_i_diff,ugriz_z_airmass_r_diff,ugriz_z_airmass_u_diff,ugriz_i_ugriz_u_diff,ugriz_i_dered_z_diff,ugriz_i_dered_i_diff,ugriz_i_dered_g_diff,ugriz_i_dered_u_diff,ugriz_i_airmass_z_diff,ugriz_i_airmass_i_diff,ugriz_i_airmass_u_diff,ugriz_r_ugriz_g_diff,ugriz_r_ugriz_u_diff,ugriz_r_dered_i_diff,ugriz_r_dered_g_diff,ugriz_r_dered_u_diff,ugriz_r_airmass_i_diff,...,ugriz_u_dered_g_diff,ugriz_u_dered_u_diff,ugriz_u_airmass_i_diff,ugriz_u_airmass_r_diff,ugriz_u_airmass_g_diff,ugriz_u_airmass_u_diff,dered_z_dered_i_diff,dered_z_dered_r_diff,dered_z_dered_g_diff,dered_z_dered_u_diff,dered_z_airmass_i_diff,dered_z_airmass_r_diff,dered_z_airmass_g_diff,dered_i_dered_r_diff,dered_i_dered_g_diff,dered_i_dered_u_diff,dered_i_airmass_z_diff,dered_i_airmass_g_diff,dered_i_airmass_u_diff,dered_r_dered_g_diff,dered_r_dered_u_diff,dered_r_airmass_i_diff,dered_r_airmass_u_diff,dered_g_dered_u_diff,dered_g_airmass_z_diff,dered_g_airmass_r_diff,dered_g_airmass_g_diff,dered_g_airmass_u_diff,dered_u_airmass_z_diff,dered_u_airmass_i_diff,dered_u_airmass_g_diff,airmass_z_airmass_i_diff,airmass_i_airmass_r_diff,airmass_i_airmass_g_diff,airmass_i_airmass_u_diff,airmass_r_airmass_g_diff,airmass_r_airmass_u_diff,airmass_g_airmass_u_diff,ugriz_dered_max_diff,ugriz_dered_min_diff,ugriz_dered_sum_diff,ugriz_airmass_max_diff,ugriz_airmass_sum_diff,dered_airmass_max_diff,ugriz_B_jester,ugriz_V_jester,ugriz_icolor,ugriz_ucolor,ugriz_p1,dered_orthogonal
count,8.0000e+04,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,8.0000e+04,8.0000e+04,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,...,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000
mean,6.0083e-02,18.2638,17.2178,16.8702,6.3499,1.1760,1.1767,1.1753,1.1756,17.7397,17.3065,17.1490,19.9839,2.9706,1.2082,90.4921,19.6729,2.8572,89.6061,1.1794,1.1727,6.7318e-03,2.6612e-03,5.8799,-0.1575,-0.5908,-2.7109,0.2787,-0.0689,-0.4704,-1.1148,-2.4860,15.9726,15.9733,15.9736,15.9730,-2.5533,0.4363,0.0887,-0.9573,-2.3285,16.1302,16.1309,16.1305,-0.6974,-2.1201,0.5219,-0.5240,-1.8952,16.5641,...,1.5961,0.2249,18.6842,18.6845,18.6831,18.6839,-0.3476,-0.7491,-1.3935,-2.7647,15.6946,15.6949,15.6935,-0.4015,-1.0460,-2.4171,16.0415,16.0411,16.0418,-0.6444,-2.0156,16.4437,16.4434,-1.3712,17.0875,17.0885,17.0870,17.0878,18.4586,18.4593,18.4582,0.0007,0.0003,-0.0011,-0.0003,-0.0014,-0.0006,0.0008,0.3109,0.1975,0.8860,18.8045,84.6122,18.4936,18.7890,18.0445,0.3100,0.0824,0.3041,0.0604
std,3.4684e-01,1.6657,1.3324,35.4367,8.8728,0.1164,0.1182,0.1148,0.1156,1.4786,1.3207,24.6431,24.6764,24.6651,11.0261,25.5608,1.9296,35.4250,36.2002,0.1185,0.1144,5.8600e-03,2.3160e-03,0.5822,24.6299,24.6382,24.6773

# feature 선택

In [27]:
# Xgboost
xgb_params = {
    'n_jobs' : -1, 
    'n_estimators': 100,
    'eval_metric': 'mlogloss',
    'eta': 0.3, # learning_rate
    'booster': 'gbtree',
    'tree_method': 'auto',
    'objective': 'multi:softmax',
    'num_class': 3,
    'random_state': 2020
}

In [28]:
xgb_clf = xgb.XGBClassifier(**xgb_params)
xgb_clf.fit(X_train,y_train)
p = xgb_clf.predict(X_val)
print(accuracy_score(p,y_val))

0.9343071777613386


In [29]:
# xgb_clf.fit(X,y)
# p = xgb_clf.predict(test)

# submission = pd.read_csv('../input/dankook/sample_submission.csv')
# submission['class'] = p
# submission.to_csv('submission.csv',index=False, encoding='utf-8-sig')